In [ ]:
# questions
#problems with reading this as $600 value for W/D - linear regression
#('w/d hookups', -358.55112862600521), ('w/d in unit', 323.94738691104828),

In [38]:
# imports
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from math import exp

# allow plots to appear directly in the notebook
import seaborn as sns
%matplotlib inline

In [181]:
data = pd.read_csv(r'C:\Users\alsherman\Desktop\GitHub\DataScience_GeneralAssembly\Data\Cleaned_Data_May_14.csv')

In [182]:
data.describe()

,ID,latitude,location_data_accuracy,longitude,average_image_size,bedroom,cat,dog,image_number,price,...,square_footage,grocery_list,gym_list,movie_theatre_list,train_station_list,airport_list,subway_station_list,Barnes_and_Nobles_list,Deloitte_list,Starbucks_list
count,3.076000e+03,3076.000000,3076.000000,3076.000000,3076.000000,3076.000000,3076,3076,3076.000000,3076.000000,...,3076.000000,3076.000000,3076.000000,3076.000000,3076.000000,3076,3076.000000,3076.000000,3076.000000,3076.000000
mean,4.984234e+09,30.585578,0.730169,-60.651097,213120.936281,2.061118,0,0,9.697009,1944.017880,...,1207.502463,9.850130,6.728869,0.997399,2.015605,0,1.365735,0.157997,0.607607,3.244148
std,1.154868e+07,15.942676,3.457620,31.613972,110118.397497,1.234268,0,0,7.893017,796.599139,...,525.660362,8.209095,7.000438,1.644105,3.758535,0,2.900269,0.488307,1.271755,4.940414
min,4.959352e+09,0.000000,0.000000,-78.269000,0.000000,0.000000,0,0,0.000000,2.000000,...,20.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000
25%,4.975371e+09,38.719200,0.000000,-77.121000,270000.000000,1.000000,0,0,3.000000,1500.000000,...,850.000000,1.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000
50%,4.985626e+09,38.855283,0.000000,-77.059200,270000.000000,2.000000,0,0,8.000000,1795.000000,...,1207.502463,9.000000,4.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,1.000000
75%,4.995176e+09,38.919865,0.000000,-76.852868,270000.000000,3.000000,0,0,15.000000,2200.000000,...,1344.000000,19.000000,13.000000,2.000000,3.000000,0,2.000000,0.000000,1.000000,4.000000
max,5.001980e+09,39.355737,22.000000,0.000000,270000.000000,6.000000,0,0,24.000000,9851.000000,...,10000.000000,20.000000,20.000000,9.000000,20.000000,0,18.000000,3.000000,6.000000,20.000000


#### transform categorical fields to numeric for scikit-learn 

In [183]:
data.country = np.where(data.country == 'US',1,0)
data.availability = np.where(data.availability == 'available now',1,0)

###Still need to remove (dummmy variable values that have only a few instances - e.g. only one person from city)

In [184]:
#Create dummy variables

data = pd.concat([data, pd.get_dummies(data.housing_type)], axis=1)
data = pd.concat([data, pd.get_dummies(data.laundry)], axis=1)
data = pd.concat([data, pd.get_dummies(data.parking)], axis=1)
data = pd.concat([data, pd.get_dummies(data.city)], axis=1)
data = pd.concat([data, pd.get_dummies(data.state)], axis=1)

data[0:1]

,ID,city,country,latitude,location_data_accuracy,longitude,state,availability,average_image_size,bathroom,...,ferry,heights,hill,marlboro,park,potomac,ridge,spring,village,washington
0,4959351766,NaN,0,0,0,0,NaN,1,270000,1,...,0,0,0,0,0,0,0,0,0,0


In [195]:
feature_cols = ['latitude',	 'location_data_accuracy',	 'longitude',	 'average_image_size',	 'bedroom',	 'cat',	 'dog',	 'image_number',	 'smoking',	 'square_footage',	 'grocery_list',	 'gym_list',	 'movie_theatre_list',	 'train_station_list',	 'airport_list',	 'subway_station_list',	 'Barnes_and_Nobles_list',	 'Deloitte_list',	 'Starbucks_list',	 'No_Housing_Data',	 'apartment',	 'condo',	 'cottage/cabin',	 'duplex',	 'flat',	 'house',	 'in-law',		
 'townhouse',	 'No_Laundry_Data',	 'laundry in bldg',	 'laundry on site',	 'w/d hookups',	 'w/d in unit',	 'No_Parking_Data',	 'attached garage',	 'carport',	 'detached garage',	 'off-street parking',	 'street parking',	 'aldie',	 'alexandria',	 'annandale',	 'arlington',	 'ashburn',	 'baltimore',	 'beltsville',	 'bethesda',	 'bladensburg',	 'bowie',	 'broadlands',	 'capitol',	 'centreville',	 'chantilly',	 'chevy',	 'clinton',	 'college',
 'columbia',	 'crofton',	 'dale',	 'damascus',	 'dumfries',	 'elkridge',	 'fairfax',	 'falls',	 'forestville',	 'frederick',	 'fredericksburg',	 'gaithersburg',	 'germantown',	 'greenbelt',	 'hamilton',	 'harpers',	 'herndon',	 'hyattsville',	 'kensington',	 'landover',	 'largo',	 'laurel',	 'laytonsville',	 'leesburg',	 'lorton',	 'manassas',	 'mclean',	 'montgomery',	 'n',
 'north',	 'northwest',	 'oak',	 'oakton',	 'oxon',	 'potomac',	 'reston',	 'riverdale',	 'rockville',	 'rosslyn',	 'silver',	 'springfield',	 'stafford',	 'sterling',	 'stone',	 'suitland',	 'takoma',	 'triangle',	 'upper',	 'vienna',	 'waldorf',	 'washingtondc',	 'wheaton',	 'woodbridge',	 'bethesda',	 'chase',	 'church',	 'college',	 'dc',
 'falls',	 'ferry',	 'heights',	 'hill',	 'marlboro',	 'park',	 'potomac',	 'ridge',	 'spring',	 'village',	 'washington']

In [199]:
### SCIKIT-LEARN ###

# create X and y
X = data[feature_cols]
y = data.price

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)

# instantiate and fit
lm2 = LinearRegression()
lm2.fit(X_train, y_train)

y_pred = lm2.predict(X_test) # Predict
# print the intercept
#print lm2.intercept_

# pair the feature names with the coefficients
#print zip(feature_cols, lm2.coef_)

# Access accuracy
y_pred = [int(y) for y in y_pred]
y_test = [int(y) for y in y_test]

print metrics.accuracy_score(y_test, y_pred)

0.0208062418726


## Logistic Regression

In [197]:
# create X and y

#feature_cols = []

X = data[feature_cols]
y = data.price

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)

# Fit model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test) # Predict

# Access accuracy
print metrics.accuracy_score(y_test, y_pred)

0.262678803641


In [89]:
con_mat = metrics.confusion_matrix(y_test, y_pred)
print con_mat

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
